In [228]:
import numpy as np
import pandas as pd

import calendar
import datetime as dt
from collections import Counter

In [229]:
deliveries = pd.read_csv(r'Einkaufslieferzeile.csv', on_bad_lines='skip', sep=';', low_memory=False,encoding='utf-8-sig')
deliveries.head()

,Attached to Line No_,Auto Charge Doc_ Line No_,Auto Charge Doc_ No_,Billing Entry No_,Billing Reference No_,Blanket Order Line No_,Blanket Order No_,Budgeted FA No_,Buy-from Vendor No_,Contract Line No_,...,Use Duplication List,Use Tax,VAT Base Amount,VAT Bus_ Posting Group,VAT Calculation Type,VAT Prod_ Posting Group,VAT _,VDP Code,Variant Code,timestamp
0,0,0,NaN,0,NaN,0,NaN,NaN,K00421,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,000000005295C63A
1,0,0,NaN,0,NaN,0,NaN,NaN,K02028,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052EC85AD
2,0,0,NaN,0,NaN,0,NaN,NaN,K04610,0,...,0,0,"0,00000000000000000000",EU,1,MWST19,"0,00000000000000000000",NaN,NaN,0000000052B4AE3C
3,0,0,NaN,0,NaN,0,NaN,NaN,K04607,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4D12C
4,0,0,NaN,0,NaN,10000,RB19/0025,NaN,K04137,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4B8F5


In [230]:
deliveries.shape

(6012, 166)

In [231]:
# drop columns where all NaN values
deliveries=deliveries.dropna(axis=1, how='all')
deliveries.shape

(6012, 129)

In [232]:
# drop columns where all 0 values
deliveries=deliveries.loc[:, (deliveries != 0).any(axis=0)]
deliveries.shape

(6012, 103)

In [233]:
# drop columns where over 50% of values are null values
deliveries=deliveries.loc[:, deliveries.isnull().mean()<0.5]
deliveries.shape

(6012, 90)

In [234]:
# get list with column names
list(deliveries.columns)

['Blanket Order Line No_',
 'Buy-from Vendor No_',
 'Cross-Reference Type No_',
 'Demand Query Line No_',
 'Document No_',
 'Item Rcpt_ Entry No_',
 'Line No_',
 'No_',
 'Order Line No_',
 'Order No_',
 'Pay-to Vendor No_',
 'Pos_ No_',
 'Prod_ Order Line No_',
 'Routing Reference No_',
 'Allow Invoice Disc_',
 'Balance',
 'Bin Code',
 'Buy-from Address',
 'Buy-from City',
 'Buy-from Contact',
 'Buy-from Post Code',
 'Buy-from Vendor Name 2',
 'Buy-from Vendor Name',
 'Contract Amount',
 'Correction',
 'Cross-Reference Type',
 'Description 2',
 'Description',
 'Dimension Set ID',
 'Direct Unit Cost Price Factor',
 'Direct Unit Cost',
 'Expected Receipt Date',
 'Expected Receipt Time',
 'FA Posting Date',
 'Gen_ Bus_ Posting Group',
 'Gen_ Prod_ Posting Group',
 'Gross Weight',
 'Indirect Cost _',
 'Item Category Code',
 'Item Charge Base Amount',
 'Job Currency Factor',
 'Job Line Amount (LCY)',
 'Job Line Amount',
 'Job Line Disc_ Amount (LCY)',
 'Job Line Discount Amount',
 'Job Line

In [235]:
# drop columns that don't seem relevant to our challenge
to_drop=['Cross-Reference Type No_', 'Demand Query Line No_', 'Document No_', 'Item Rcpt_ Entry No_',
 'Line No_', 'Order Line No_', 'Pay-to Vendor No_', 'Pos_ No_', 'Prod_ Order Line No_', 'Routing Reference No_', 'Allow Invoice Disc_', 'Balance',
 'Bin Code', 'Contract Amount', 'Correction', 'Cross-Reference Type', 'Description 2', 'Dimension Set ID', 'Direct Unit Cost Price Factor',
 'Direct Unit Cost', 'Gen_ Bus_ Posting Group', 'Gen_ Prod_ Posting Group', 'Gross Weight', 'Indirect Cost _', 'Item Charge Base Amount',
 'Job Currency Factor', 'Job Line Amount (LCY)', 'Job Line Amount', 'Job Line Disc_ Amount (LCY)', 'Job Line Discount Amount', 'Job Line Discount _',
 'Job Total Price (LCY)', 'Job Unit Price (LCY)', 'Job Unit Price', 'Line Discount _', 'Net Weight', 'Overhead Rate', 'Pack Sample Quantity', 'Price Factor', 
 'Qty_ per Unit of Measure', 'Salvage Value', 'Shortcut Dimension 1 Code', 'Shortcut Dimension 2 Code','Qty_ Invoiced (Base)','Quantity (Base)',
 'Type', 'Unit Cost (LCY)', 'Unit Cost', 'Unit Price (LCY)', 'Unit Volume', 'Unit of Measure (Cross Ref_)', 'Unit of Measure Code',
 'Unit of Measure', 'Units per Parcel', 'VAT Base Amount', 'VAT Bus_ Posting Group', 'VAT Calculation Type', 'VAT Prod_ Posting Group','Blanket Order Line No_',
 'VAT _', 'Buy-from Vendor Name 2','Buy-from Address', 'timestamp','Item Category Code','Posting Group','Description','Buy-from Contact','Job Total Price','Quantity']
deliveries.drop(to_drop, inplace=True, axis=1)

In [236]:
deliveries.head()

,Buy-from Vendor No_,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,...,Order Date,Order Quantity,Original Date,Planned Receipt Date,Posting Date,Promised Receipt Date,Qty_ Rcd_ Not Invoiced,Quantity Invoiced,Really Receipt DateTime,Requested Receipt Date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,...,23.09.2019 00:00:00,"0,00000000000000000000",17.12.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,...,15.11.2019 00:00:00,"0,00000000000000000000",17.12.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,...,19.12.2019 00:00:00,"0,00000000000000000000",19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,"0,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00
3,K04607,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,06.01.2020 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,...,03.12.2019 00:00:00,"0,00000000000000000000",03.12.2019 00:00:00,06.01.2020 00:00:00,30.12.2019 00:00:00,06.01.2020 00:00:00,"0,00000000000000000000","300,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00
4,K04137,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,18.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,8,...,03.07.2019 00:00:00,"0,00000000000000000000",18.12.2019 00:00:00,18.12.2019 00:00:00,30.12.2019 00:00:00,18.12.2019 00:00:00,"0,00000000000000000000","8704,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00


In [237]:
dates=deliveries[[
 'Expected Receipt Date', 'Expected Receipt Time', 'FA Posting Date', 'Lead Time Calculation',  'Planned Receipt Date',
 'Order Date', 'Original Date', 'Posting Date', 'Promised Receipt Date', 'Really Receipt DateTime', 'Requested Receipt Date']]
dates.head(3)

,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,Planned Receipt Date,Order Date,Original Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,19.12.2019 00:00:00,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00


**Date columns to keep**

Order Date: date the order was created. 

Posting Date: date you want as the Posting Date on the Ledger Entry tables when the order is Shipped and Invoiced.

Promised Receipt Date: date your vendor promised they would deliver the goods. NAV does not change this date, so it allows you to set a marker to measure your vendor’s performance against what was promised.

Requested Receipt Date: date you wanted to receive the goods from the vendor. This date is not recalculated in NAV, so once it has been established, it gives you a marker to measure your vendor’s performance against the requested date.

Really Receipt DateTime

In [238]:
# searching for how Microsoft Nav calculates dates, we'll drop some columns indicating dates, taking into consideration the following:
# requested receipt date - lead time calculation = order date
# requested receipt date + inbound whse. handling time + safety lead time = expected receipt date
# lead time calculation: amount of time the product needs in order to be delivered to your company that the Purchasing Agent needs to be alerted
# Expected Receipt Date: date you currently expect your vendor to deliver the order
# Planned Receipt Date: date the order is expected to arrive at the warehouse location, uses the Requested Receipt Date
to_drop=['Expected Receipt Time','FA Posting Date','Lead Time Calculation','Planned Receipt Date','Original Date',]
deliveries.drop(to_drop, inplace=True, axis=1)
deliveries.head()


,Buy-from Vendor No_,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Location Code,Order Date,Order Quantity,Posting Date,Promised Receipt Date,Qty_ Rcd_ Not Invoiced,Quantity Invoiced,Really Receipt DateTime,Requested Receipt Date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,B-01,23.09.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,B-01,15.11.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,B-01,19.12.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,19.12.2019 00:00:00,"0,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00
3,K04607,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,06.01.2020 00:00:00,B-01,03.12.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,06.01.2020 00:00:00,"0,00000000000000000000","300,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00
4,K04137,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,18.12.2019 00:00:00,B-01,03.07.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,18.12.2019 00:00:00,"0,00000000000000000000","8704,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00


Qty. Rcd. Not Invoiced Field: specifies how many units, such as pieces, of the ordered item have been received but not yet invoiced.

Quantity (Base): Inventory is stored in the base unit of measure, and every time that you sell or purchase or use any other unit of measure, NAV needs to know the quantity expressed in base unit of measure, so it can update inventory correctly

Order Quantity: covers all gross requirements (forecast, sales orders, and also replenishment of the inventory level).

In [239]:
deliveries.head(3)

,Buy-from Vendor No_,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Location Code,Order Date,Order Quantity,Posting Date,Promised Receipt Date,Qty_ Rcd_ Not Invoiced,Quantity Invoiced,Really Receipt DateTime,Requested Receipt Date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,B-01,23.09.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,B-01,15.11.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,B-01,19.12.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,19.12.2019 00:00:00,"0,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00


In [240]:
deliveries.columns = deliveries.columns.str.replace(" ", "_").str.lower()
deliveries=deliveries.rename(columns={'buy-from_vendor_no_': 'vendor_no',"no_": "delivery_no","order_no_": "order_no",'buy-from_city':'vendor_city',
'buy-from_post_code':'vendor_postcode','buy-from_vendor_name':'vendor_name','expected_receipt_date':'expected_date','order_quantity':'order_qty',
 'promised_receipt_date': 'promised_date', 'qty__rcd__not_invoiced':'qty_rcd_not_invoiced','quantity_invoiced':'qty_invoiced','really_receipt_datetime':'delivery_date',
 'requested_receipt_date':'requested_date'})
deliveries.buy_from_vendor_name=deliveries.vendor_name.str.replace(",", " ") # commas would create additional columns when exporting to csv
list(deliveries.columns)

C:\Users\virgi\AppData\Local\Temp/ipykernel_16692/1221190492.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  deliveries.buy_from_vendor_name=deliveries.vendor_name.str.replace(",", " ") # commas would create additional columns when exporting to csv


['vendor_no',
 'delivery_no',
 'order_no',
 'vendor_city',
 'vendor_postcode',
 'vendor_name',
 'expected_date',
 'location_code',
 'order_date',
 'order_qty',
 'posting_date',
 'promised_date',
 'qty_rcd_not_invoiced',
 'qty_invoiced',
 'delivery_date',
 'requested_date']

In [243]:
deliveries.qty_invoiced = deliveries.qty_invoiced.map(lambda x: str(x)[:-21])
deliveries.order_qty = deliveries.order_qty.map(lambda x: str(x)[:-21])
deliveries.qty_rcd_not_invoiced = deliveries.qty_rcd_not_invoiced.map(lambda x: str(x)[:-21])
deliveries.head(3)

,vendor_no,delivery_no,order_no,vendor_city,vendor_postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,B-01,23.09.2019 00:00:00,0,30.12.2019 00:00:00,17.12.2019 00:00:00,0,22730,16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,B-01,15.11.2019 00:00:00,0,30.12.2019 00:00:00,17.12.2019 00:00:00,0,15,01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,B-01,19.12.2019 00:00:00,0,30.12.2019 00:00:00,19.12.2019 00:00:00,0,2000,01.01.1753 00:00:00,19.12.2019 00:00:00


In [245]:
deliveries = deliveries.astype({'qty_invoiced': 'int64','order_qty': 'int64','qty_rcd_not_invoiced': 'int64'})
deliveries.dtypes

vendor_no               object
delivery_no             object
order_no                object
vendor_city             object
vendor_postcode         object
vendor_name             object
expected_date           object
location_code           object
order_date              object
order_qty                int64
posting_date            object
promised_date           object
qty_rcd_not_invoiced     int64
qty_invoiced             int64
delivery_date           object
requested_date          object
dtype: object

In [246]:
deliveries.head(3)

,vendor_no,delivery_no,order_no,vendor_city,vendor_postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,B-01,23.09.2019 00:00:00,0,30.12.2019 00:00:00,17.12.2019 00:00:00,0,22730,16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,B-01,15.11.2019 00:00:00,0,30.12.2019 00:00:00,17.12.2019 00:00:00,0,15,01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,B-01,19.12.2019 00:00:00,0,30.12.2019 00:00:00,19.12.2019 00:00:00,0,2000,01.01.1753 00:00:00,19.12.2019 00:00:00


In [247]:
deliveries.expected_date = deliveries.expected_date.map(lambda x: str(x)[:-9])
deliveries.order_date = deliveries.order_date.map(lambda x: str(x)[:-9])
deliveries.posting_date = deliveries.posting_date.map(lambda x: str(x)[:-9])
deliveries.promised_date = deliveries.promised_date.map(lambda x: str(x)[:-9])
deliveries.delivery_date = deliveries.delivery_date.map(lambda x: str(x)[:-9])
deliveries.requested_date = deliveries.requested_date.map(lambda x: str(x)[:-9])

In [248]:
deliveries.head(3)

,vendor_no,delivery_no,order_no,vendor_city,vendor_postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019,B-01,23.09.2019,0,30.12.2019,17.12.2019,0,22730,16.12.2019,17.12.2019
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019,B-01,15.11.2019,0,30.12.2019,17.12.2019,0,15,01.01.1753,17.12.2019
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019,B-01,19.12.2019,0,30.12.2019,19.12.2019,0,2000,01.01.1753,19.12.2019


In [249]:
deliveries.shape

(6012, 16)

In [250]:
# Some dates with a value of 01.01.1753 an 09.09.2099 mean the field is blank or there's been an error in the input.
# we will filter out these rows (1,236 from 6012)
deliveries = deliveries[(deliveries['expected_date']!= '01.01.1753')&(deliveries['expected_date']!= '09.09.2099')]
deliveries = deliveries[(deliveries['delivery_date']!= '01.01.1753')&(deliveries['delivery_date']!= '09.09.2099')]
deliveries = deliveries[(deliveries['promised_date']!= '01.01.1753')&(deliveries['promised_date']!= '09.09.2099')]
deliveries = deliveries[(deliveries['order_date']!= '01.01.1753')&(deliveries['order_date']!= '09.09.2099')]
deliveries = deliveries[(deliveries['posting_date']!= '01.01.1753')&(deliveries['posting_date']!= '09.09.2099')]
deliveries = deliveries[(deliveries['requested_date']!= '01.01.1753')&(deliveries['requested_date']!= '09.09.2099')]
deliveries.shape

(4207, 16)

In [252]:
deliveries = deliveries.astype({'order_date': 'datetime64','posting_date': 'datetime64','promised_date': 'datetime64',
'delivery_date': 'datetime64','requested_date': 'datetime64','expected_date':'datetime64'})
deliveries.dtypes

vendor_no                       object
delivery_no                     object
order_no                        object
vendor_city                     object
vendor_postcode                 object
vendor_name                     object
expected_date           datetime64[ns]
location_code                   object
order_date              datetime64[ns]
order_qty                        int64
posting_date            datetime64[ns]
promised_date           datetime64[ns]
qty_rcd_not_invoiced             int64
qty_invoiced                     int64
delivery_date           datetime64[ns]
requested_date          datetime64[ns]
dtype: object

In [253]:
deliveries.isna().sum()

vendor_no               0
delivery_no             0
order_no                0
vendor_city             2
vendor_postcode         4
vendor_name             0
expected_date           0
location_code           0
order_date              0
order_qty               0
posting_date            0
promised_date           0
qty_rcd_not_invoiced    0
qty_invoiced            0
delivery_date           0
requested_date          0
dtype: int64

In [254]:
# drop rows with nan values
deliveries = deliveries.dropna()
deliveries.shape

(4202, 16)

In [255]:
# drop duplicates
deliveries.drop_duplicates(keep='first')
deliveries.shape

(4202, 16)

In [256]:
# export as csv file
deliveries.to_csv(r'deliveries_clean.csv', index=False);